In [2]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())
    

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/2D Klein Gordon/stan'

In [4]:
# !pip install smt

In [3]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1]).reshape(-1,1)
b_value = np.array([0.0,0.25,0.5,1.0]).reshape(-1,1)


LR_tune, B_value = np.meshgrid(lr_tune,b_value)

LR_tune = LR_tune.flatten('F').reshape(-1,1)
B_value = B_value.flatten('F').reshape(-1,1)


lrb_tune = np.hstack((LR_tune,B_value))

In [6]:
def true_2D_1(xt): #True function for 2D_1 Klein Gordon Equation x \in [-50,50] , t \in [0,10]
    y = xt[:,0]*np.cos(xt[:,1])
    return y.reshape(-1,1)    

In [7]:
loss_thresh = 0.1

x = np.linspace(-5,5,500).reshape(-1,1)
t = np.linspace(0,10,1000).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

y_true = true_2D_1(xt)
y_true_norm = np.linalg.norm(y_true,2)

#bound_pts_idx = ((X == -5) + (X == 5) + (T == 0)).reshape(-1,)

#xt_bound = xt[bound_pts_idx,:]
#y_bound = y_true[bound_pts_idx,:]


xt_test_tensor = torch.from_numpy(xt).float().to(device)


lb_xt = xt[0]
ub_xt = xt[-1]

In [8]:
def trainingdata(N_I,N_B,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    x_BC1 = np.random.uniform(size = N_I).reshape(-1,1)
    t_BC1 = np.zeros((N_I,1))
    samples = np.hstack((x_BC1,t_BC1))
    xt_BC1 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC1 = true_2D_1(xt_BC1)
    
    x_BC2 = np.zeros((int(N_B/2),1))
    t_BC2 = np.random.uniform(size = int(N_B/2)).reshape(-1,1)
    samples = np.hstack((x_BC2,t_BC2))
    xt_BC2 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC2 = true_2D_1(xt_BC2)
    
    x_BC3 = np.ones((int(N_B/2),1))
    t_BC3 = np.random.uniform(size = int(N_B/2)).reshape(-1,1)
    samples = np.hstack((x_BC3,t_BC3))
    xt_BC3 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC3 = true_2D_1(xt_BC3)

    xt_BC = np.vstack((xt_BC1,xt_BC2,xt_BC3))
    y_BC = np.vstack((y_BC1,y_BC2,y_BC3))

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll, xt_BC)) # append training points to collocation points 

    return xt_coll, xt_BC, y_BC

In [9]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,beta_init):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            
        
        self.beta = Parameter(beta_init*torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True

            
    'foward pass'
    def forward(self,xt):
        if torch.is_tensor(xt) != True:         
            xt = torch.from_numpy(xt)                
        
        ubxt = torch.from_numpy(ub_xt).float().to(device)
        lbxt = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xt = (xt - lbxt)/(ubxt - lbxt)
        
        #convert to float
        a = xt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(z) + self.beta[:,i]*z*self.activation(z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC(self,xt,y):
                
        loss_bc = self.loss_function(self.forward(xt), y)
                
        return loss_bc
    
    def loss_PDE(self, xt_coll, f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        y = self.forward(g) 
        
        
        y_x_t = autograd.grad(y,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        y_xx_tt = autograd.grad(y_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        #du_dx = u_x_t[:,[0]]
        
        d2y_dx2 = y_xx_tt[:,[0]]
        d2y_dt2 = y_xx_tt[:,[1]]    
        

        f = d2y_dt2 - d2y_dx2 + torch.pow(y,2) + (g[:,0]*torch.cos(g[:,1])).reshape(-1,1) - (torch.pow(g[:,0],2)*torch.pow(torch.cos(g[:,1]),2)).reshape(-1,1)
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_BC,y_BC,xt_coll,f_hat):

        loss_BC = self.loss_BC(xt_BC,y_BC)
        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_val = loss_BC + loss_f
        
        return loss_val
         
    'test neural network'
    def test(self):
        y_pred = self.forward(xt_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()
   
        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re  

In [10]:
def data_update(loss_np):
    train_loss.append(loss_np)
    beta_val.append(PINN.beta.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [11]:
def train_step(xt_BC, y_BC, xt_coll, f_hat,seed):
    # x_coll_np_array = colloc_pts(N_f,seed*123)
    # x_coll_train = torch.from_numpy(x_coll_np_array).float().to(device)        
    
    # f_hat = torch.zeros(x_coll_train.shape[0],1).to(device)
    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_BC, y_BC, xt_coll,f_hat)
        loss.backward()
        #print(loss.cpu().detach().numpy())
        return loss

    optimizer.step(closure)

In [12]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0

    xt_coll, xt_BC, y_BC = trainingdata(N_I,N_B,N_f,rep*11)
    xt_coll = torch.from_numpy(xt_coll).float().to(device)
    xt_BC = torch.from_numpy(xt_BC).float().to(device)
    y_BC = torch.from_numpy(y_BC).float().to(device)

    f_hat = torch.zeros(xt_coll.shape[0],1).to(device)
    
    nan_flag = 0
    for i in range(max_iter):
        train_step(xt_BC, y_BC, xt_coll,f_hat,i)
        loss_np = PINN.loss(xt_BC, y_BC, xt_coll,f_hat).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1          
                thresh_flag = 1       
        data_update(loss_np)
        
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])   
        
        if(np.isnan(loss_np)):
            nan_flag =1
            print("NAN BREAK!")
            break
            
         

    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))
    
    return nan_flag

In [13]:
nan_tune = []
for tune_reps in range(20):
    
    max_reps = 10 #10
    max_iter = 100 #100
    label = "KG_stan_tune"+str(tune_reps)

    train_loss_full = []
    test_mse_full = []
    test_re_full = []
    beta_full = []
    elapsed_time= np.zeros((max_reps,1))
    time_threshold = np.empty((max_reps,1))
    time_threshold[:] = np.nan
    epoch_threshold = max_iter*np.ones((max_reps,1))
    
    beta_init = lrb_tune[tune_reps,1]

    N_I = 200  #Total number of data points for 'y'
    N_B = 400
    N_f = 10000 #Total number of collocation points
    
    for reps in range(max_reps):
        print(label)
        print(reps)
        train_loss = []
        test_mse_loss = []
        test_re_loss = []
        beta_val = []
      
        torch.manual_seed(reps*36)

        layers = np.array([2,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
        #layers = np.array([2,50,50,50,50,50,50,50,1])

        PINN = Sequentialmodel(layers,beta_init)
    
        PINN.to(device)

        'Neural Network Summary'
        print(PINN)

        params = list(PINN.parameters())
      

        optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lrb_tune[tune_reps,0], 
                                max_iter = 10, 
                                max_eval = 15, 
                                tolerance_grad = 1e-8, 
                                tolerance_change = 1e-8, 
                                history_size = 100, 
                                line_search_fn = 'strong_wolfe')



          
        nan_flag = train_model(max_iter,reps)
    
        
      
        torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
        train_loss_full.append(train_loss)
        test_mse_full.append(test_mse_loss)
        test_re_full.append(test_re_loss)
        #elapsed_time[reps] = time.time() - start_time
        beta_full.append(beta_val)
        
        if(nan_flag == 1):
            nan_tune.append(tune_reps)
            break


      #print('Training time: %.2f' % (elapsed_time[reps]))
      
    mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "beta": beta_full, "label": label}
    savemat(label+'.mat', mdic)

KG_stan_tune0
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 58.700226 Test MSE 8.644311399021719 Test RE 1.405313641281015
1 Train Loss 57.910076 Test MSE 8.487540787907289 Test RE 1.3925121636877191
2 Train Loss 56.02439 Test MSE 9.022194644527383 Test RE 1.4357015076399968
3 Train Loss 47.836292 Test MSE 7.362302639044134 Test 

97 Train Loss 3.9114406 Test MSE 2.148736721644375 Test RE 0.7006476395798695
98 Train Loss 3.903901 Test MSE 2.1467923582502837 Test RE 0.7003305644507407
99 Train Loss 3.889979 Test MSE 2.147132654097697 Test RE 0.7003860682209369
Training time: 69.24
KG_stan_tune0
1
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
1
0 Train Loss 57.67988 Test MSE

94 Train Loss 6.1673527 Test MSE 4.597209431699773 Test RE 1.0248379635276157
95 Train Loss 5.2581654 Test MSE 4.09413780543741 Test RE 0.967139872838781
96 Train Loss 4.8376303 Test MSE 3.9350032828371684 Test RE 0.948157773088217
97 Train Loss 4.2011375 Test MSE 3.674093960724093 Test RE 0.9161850263726089
98 Train Loss 3.4738383 Test MSE 3.1430948421643388 Test RE 0.8473967680023649
99 Train Loss 3.1987507 Test MSE 3.1155565592629197 Test RE 0.8436763598333138
Training time: 68.21
KG_stan_tune0
2
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_featur

91 Train Loss 6.8284016 Test MSE 3.1198199516849563 Test RE 0.8442534146300705
92 Train Loss 6.7430315 Test MSE 3.124723592532712 Test RE 0.8449166404126269
93 Train Loss 6.5699534 Test MSE 3.0020195926702424 Test RE 0.8281610890976165
94 Train Loss 6.506839 Test MSE 3.046282979449123 Test RE 0.8342441736922716
95 Train Loss 6.302414 Test MSE 2.9724999832244086 Test RE 0.8240792723726023
96 Train Loss 6.186656 Test MSE 2.82306910422307 Test RE 0.8030985001645619
97 Train Loss 6.1136637 Test MSE 2.7614061690093914 Test RE 0.7942792291402417
98 Train Loss 5.948027 Test MSE 2.6331814102187305 Test RE 0.7756190200308102
99 Train Loss 5.689459 Test MSE 2.5320196742391956 Test RE 0.7605742156393626
Training time: 74.48
KG_stan_tune0
3
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50,

88 Train Loss 0.9454146 Test MSE 6.144603111832836 Test RE 1.184827098552059
89 Train Loss 0.9322938 Test MSE 6.158020918065085 Test RE 1.1861200307977182
90 Train Loss 0.9222169 Test MSE 6.171647648396795 Test RE 1.1874316539950243
91 Train Loss 0.91090405 Test MSE 6.219137066745958 Test RE 1.1919914068567554
92 Train Loss 0.9031901 Test MSE 6.245146094983522 Test RE 1.194481317557562
93 Train Loss 0.89591014 Test MSE 6.246753284199899 Test RE 1.1946350076367829
94 Train Loss 0.8887078 Test MSE 6.225235148803022 Test RE 1.1925756583664082
95 Train Loss 0.8815945 Test MSE 6.22579877095965 Test RE 1.1926296440259885
96 Train Loss 0.8723711 Test MSE 6.239588555773837 Test RE 1.1939497163602
97 Train Loss 0.8656387 Test MSE 6.24525513707982 Test RE 1.194491745510434
98 Train Loss 0.8594803 Test MSE 6.250041667478823 Test RE 1.1949494030312033
99 Train Loss 0.85336137 Test MSE 6.244871053422523 Test RE 1.194455014279908
Training time: 85.42
KG_stan_tune0
4
Sequentialmodel(
  (activation): 

...

80 Train Loss 0.47198805 Test MSE 3.111598515309726 Test RE 0.8431402807571698
81 Train Loss 0.46321285 Test MSE 3.1181732137882783 Test RE 0.8440305736298874
82 Train Loss 0.4576016 Test MSE 3.145567589806759 Test RE 0.8477300360872542
83 Train Loss 0.45161748 Test MSE 3.1735127733489166 Test RE 0.8514873211958822
84 Train Loss 0.44633555 Test MSE 3.1936663055546073 Test RE 0.8541867460840671
85 Train Loss 0.44192794 Test MSE 3.199892892015132 Test RE 0.8550190305551792
86 Train Loss 0.43655363 Test MSE 3.198888218063475 Test RE 0.8548847943769308
87 Train Loss 0.4307018 Test MSE 3.220702345256465 Test RE 0.8577946930426027
88 Train Loss 0.42829737 Test MSE 3.19662929150161 Test RE 0.8545828983933369
89 Train Loss 0.42243132 Test MSE 3.216907636501187 Test RE 0.8572892069635163
90 Train Loss 0.42043605 Test MSE 3.2115854436066043 Test RE 0.8565797449941455
91 Train Loss 0.41188675 Test MSE 3.249754892462366 Test RE 0.8616549026451762
92 Train Loss 0.40735477 Test MSE 3.266282896788082

76 Train Loss 0.57554096 Test MSE 6.861152182717804 Test RE 1.2520065080853104
77 Train Loss 0.5694175 Test MSE 6.863567231521556 Test RE 1.2522268348403949
78 Train Loss 0.56341517 Test MSE 6.873383102409281 Test RE 1.253121945570891
79 Train Loss 0.5574827 Test MSE 6.894862644646105 Test RE 1.2550784413336626
80 Train Loss 0.5517772 Test MSE 6.904476152814164 Test RE 1.2559531145936096
81 Train Loss 0.5470417 Test MSE 6.933563227788862 Test RE 1.2585958645331314
82 Train Loss 0.5419636 Test MSE 6.938262186417485 Test RE 1.2590222750258597
83 Train Loss 0.5388056 Test MSE 6.95229705666263 Test RE 1.260295020772029
84 Train Loss 0.53575635 Test MSE 6.974256629412294 Test RE 1.2622838397463987
85 Train Loss 0.5313823 Test MSE 6.988148707821071 Test RE 1.2635403910388676
86 Train Loss 0.52801144 Test MSE 6.981787654273547 Test RE 1.2629651830471726
87 Train Loss 0.52419496 Test MSE 6.988263750356408 Test RE 1.2635507915253141
88 Train Loss 0.520798 Test MSE 7.006584011650686 Test RE 1.26

69 Train Loss 0.0018204184 Test MSE 0.0012265147855314031 Test RE 0.016739576030515388
70 Train Loss 0.0016593256 Test MSE 0.001134435831548561 Test RE 0.016098967660641374
71 Train Loss 0.0016026039 Test MSE 0.0011032511723387233 Test RE 0.015876152396539264
72 Train Loss 0.001404254 Test MSE 0.0010867818857427628 Test RE 0.01575720756256763
73 Train Loss 0.0013730002 Test MSE 0.001021482562116543 Test RE 0.01527648854848051
74 Train Loss 0.0012904893 Test MSE 0.0010038524654956626 Test RE 0.015144083839343807
75 Train Loss 0.0012329813 Test MSE 0.0010320133597995923 Test RE 0.015355031792917683
76 Train Loss 0.0011984578 Test MSE 0.0009432103601331025 Test RE 0.014679536443365542
77 Train Loss 0.0011543055 Test MSE 0.0008670105445165997 Test RE 0.014074087604042029
78 Train Loss 0.0010545107 Test MSE 0.0007701476847820736 Test RE 0.013264627769609488
79 Train Loss 0.0010000562 Test MSE 0.0007120822912656986 Test RE 0.012754785312825086
80 Train Loss 0.0009522224 Test MSE 0.0005825725

61 Train Loss 0.26466894 Test MSE 3.62431448462446 Test RE 0.9099572669521909
62 Train Loss 0.26319638 Test MSE 3.627551927618727 Test RE 0.9103635889223425
63 Train Loss 0.2612522 Test MSE 3.6348878479400892 Test RE 0.9112836283761302
64 Train Loss 0.25989777 Test MSE 3.6483052665116 Test RE 0.9129639843808024
65 Train Loss 0.25810558 Test MSE 3.65937681132878 Test RE 0.9143482249932976
66 Train Loss 0.2559958 Test MSE 3.669324723424145 Test RE 0.9155901963612504
67 Train Loss 0.2543368 Test MSE 3.68704592923715 Test RE 0.9177984799522876
68 Train Loss 0.25282988 Test MSE 3.7019853081367535 Test RE 0.9196559938648916
69 Train Loss 0.2514402 Test MSE 3.7238414878532384 Test RE 0.9223667807645113
70 Train Loss 0.24995215 Test MSE 3.7405270041565553 Test RE 0.9244309079221207
71 Train Loss 0.24868126 Test MSE 3.7413665654152917 Test RE 0.924534646356687
72 Train Loss 0.2472467 Test MSE 3.752267008592186 Test RE 0.9258804791492746
73 Train Loss 0.2458246 Test MSE 3.767313627532342 Test RE

57 Train Loss 0.927692 Test MSE 6.072863528915484 Test RE 1.1778902342221922
58 Train Loss 0.9149302 Test MSE 6.098621182399971 Test RE 1.1803855632745897
59 Train Loss 0.90252936 Test MSE 6.142146359817378 Test RE 1.1845902144668565
60 Train Loss 0.8865953 Test MSE 6.150890213852782 Test RE 1.1854330957347117
61 Train Loss 0.87016714 Test MSE 6.134917886322398 Test RE 1.1838929582263733
62 Train Loss 0.8568356 Test MSE 6.161889403581725 Test RE 1.1864925342222967
63 Train Loss 0.8459983 Test MSE 6.177979055935741 Test RE 1.1880405826766836
64 Train Loss 0.83392334 Test MSE 6.231117094599881 Test RE 1.1931389310007023
65 Train Loss 0.8212979 Test MSE 6.237404847289504 Test RE 1.193740770989841
66 Train Loss 0.81256443 Test MSE 6.229062061370888 Test RE 1.1929421651311432
67 Train Loss 0.8002411 Test MSE 6.279149235066546 Test RE 1.1977287179380955
68 Train Loss 0.79111236 Test MSE 6.282500032139358 Test RE 1.1980482525163882
69 Train Loss 0.78350395 Test MSE 6.316490765011846 Test RE 1

In [14]:
nan_tune

[17]

In [4]:
import scipy.io as sio
import numpy as np

In [5]:
for tune_reps in range(20):
    label = "KG_stan_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps," ",np.mean(re[:,-1]))

0   0.7464338405153359
1   0.7088968006531634
2   0.698321253106821
3   0.6710382757272744
4   0.34801528958135347
5   0.6235461617123761
6   0.5308507424773917
7   0.5845655022100533
8   0.6277262044572863
9   0.7934249883767227
10   0.3782206980277623
11   0.5431666081727968
12   0.38807631804346154
13   0.7125802670146782
14   0.493512693155249
15   0.7197849631318884
16   0.33691845060768677
17   [[1.38699518 1.38027319 1.48567026 1.39873719 1.40199089 1.38184738
  1.38987924 1.37999992 1.37952436 1.38065303 1.3875345  1.40504569
  1.41214976 1.41156113 1.44304222 1.45311749 1.45346139 1.39357478
         nan]]
18   0.578231548565742
19   0.6890833507252422


In [6]:
lrb_tune[4]

array([0.1, 0. ])